Hola, Luis!

Mi nombre es Tonatiuh Cruz. Me complace revisar tu proyecto hoy.

Al identificar cualquier error inicialmente, simplemente los destacaré. Te animo a localizar y abordar los problemas de forma independiente como parte de tu preparación para un rol como data-scientist. En un entorno profesional, tu líder de equipo seguiría un enfoque similar. Si encuentras la tarea desafiante, proporcionaré una pista más específica en la próxima iteración.

Encontrarás mis comentarios a continuación - **por favor no los muevas, modifiques o elimines**.

Puedes encontrar mis comentarios en cajas verdes, amarillas o rojas como esta:

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Éxito. Todo está hecho correctamente.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Observaciones. Algunas recomendaciones.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Necesita corrección. El bloque requiere algunas correcciones. El trabajo no puede ser aceptado con comentarios en rojo.
</div>

Puedes responderme utilizando esto:



<div class="alert alert-block alert-info">
<b>Respuesta del estudiante.</b> <a class="tocSkip"></a>



<div class="alert alert-block alert-success">
<b>Resumen de la revisión</b> <a class="tocSkip"></a>

Hola Luis, tu código está bien ordenado y tus resultados son correctos dados el dataset. Además entrenaras los modelos con un más de hiperparámetros y realias una conclusión al final del notebook de tus interpretaciones de los resultados y el modelo seleccionado. Sigue con el excelente trabajo!
</div>

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.metrics import accuracy_score

import numpy as np

In [3]:
df = pd.read_csv('/datasets/users_behavior.csv') 
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


# Visualizacion y limpieza de datos

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Buen trabajo cargando las librerías y el dataset para el proyecto! Además usas el método info() , los cuales son muy importantes para conocer la estructura de nuestros datos.
</div>

In [5]:
#Buscar duplicados
df.duplicated().sum()

0

In [6]:
#Buscar valores faltantes
df.isnull().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [8]:
#Corrección en tipos de datos
df['calls'] = df['calls'].astype('int64')
df['messages'] = df['messages'].astype('int64')
df['is_ultra'] = df['is_ultra'].astype('bool')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   bool   
dtypes: bool(1), float64(2), int64(2)
memory usage: 103.7 KB


In [9]:
#Revisar si hay valores extremos
df.describe()


,calls,minutes,messages,mb_used
count,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836
std,33.236368,234.569872,36.148326,7570.968246
min,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500
50%,62.000000,430.600000,30.000000,16943.235000
75%,82.000000,571.927500,57.000000,21424.700000
max,244.000000,1632.060000,224.000000,49745.730000


### Observaciones:
Valores en cero: aparecen en todas las columnas. No son missing values, pero conviene evaluar si tienen sentido (ej. alguien que nunca llama ni manda mensajes pero sí usa datos, o viceversa).

Outliers: los máximos están bastante alejados de la media en minutes, messages y mb_used. No parecen imposibles, pero habría que confirmar si son casos reales o errores de registro.

Distribuciones: messages y mb_used probablemente tengan sesgo a la derecha (muchos usuarios con valores bajos y unos pocos con valores muy altos).

Coherencia: sería bueno cruzar variables. Ejemplo: usuarios con calls=0 y minutes>0 no deberían existir (sería inconsistente).

In [12]:
usuarios_sin_llamadas = (df['calls'] == 0).sum()
total_usuarios = len(df)
print("Usuarios sin llamadas:", usuarios_sin_llamadas)
print("Porcentaje:", usuarios_sin_llamadas / total_usuarios * 100, "%")


Usuarios sin llamadas: 40
Porcentaje: 1.2445550715619167 %


In [13]:
sin_llamadas_y_mensajes = df[(df['calls'] == 0) & (df['messages'] == 0)]

# número de usuarios
print("Usuarios sin llamadas y sin mensajes:", sin_llamadas_y_mensajes.shape[0])


Usuarios sin llamadas y sin mensajes: 1


 Solo hay un usuario sin llamadas y sin mensajes. Por lo que no creo que afece a los resultados 

In [10]:
#Revisar si hay valores imposibles
(df < 0).sum()


calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [11]:
#Revisando balance entre planes ultra = 1, samrt = 0
df['is_ultra'].value_counts(normalize=True)

False    0.693528
True     0.306472
Name: is_ultra, dtype: float64

# 2. Segmentación de datos fuente

In [16]:
# Separar features y target
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

# Primero dividir en train+valid vs test (20% test)
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345
)

# Ahora dividir train+valid en train vs valid (25% de 80% = 20% total)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, target_train_valid, test_size=0.25, random_state=12345
)

# Revisar tamaños
print("Tamaños de los conjuntos:")
print("Train:", features_train.shape)
print("Valid:", features_valid.shape)
print("Test:", features_test.shape)

Tamaños de los conjuntos:
Train: (1928, 4)
Valid: (643, 4)
Test: (643, 4)


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Muy bien, dividiste adecuadamente los datos en entrenamiento, validación. Los dos conjuntos son muy importantes, el de validación en particular nos es útil para probar los diferentes hiperparámetros antes de hacer el testeo final con el set de prueba.
</div>

# 3. Modelos

In [19]:
# Árbol de decisión
for depth in range(1, 11):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    preds = model.predict(features_valid)
    print("Decision Tree depth =", depth, "Accuracy:", accuracy_score(target_valid, preds))

# Bosque aleatorio
for est in [10, 50, 100]:
    model = RandomForestClassifier(n_estimators=est, random_state=12345)
    model.fit(features_train, target_train)
    preds = model.predict(features_valid)
    print("Random Forest n_estimators =", est, "Accuracy:", accuracy_score(target_valid, preds))

# Regresión logística
model = LogisticRegression(random_state=12345, max_iter=1000)
model.fit(features_train, target_train)
preds = model.predict(features_valid)
print("Logistic Regression Accuracy:", accuracy_score(target_valid, preds))

Decision Tree depth = 1 Accuracy: 0.7387247278382582
Decision Tree depth = 2 Accuracy: 0.7573872472783826
Decision Tree depth = 3 Accuracy: 0.7651632970451011
Decision Tree depth = 4 Accuracy: 0.7636080870917574
Decision Tree depth = 5 Accuracy: 0.7589424572317263
Decision Tree depth = 6 Accuracy: 0.7573872472783826
Decision Tree depth = 7 Accuracy: 0.7744945567651633
Decision Tree depth = 8 Accuracy: 0.7667185069984448
Decision Tree depth = 9 Accuracy: 0.7620528771384136
Decision Tree depth = 10 Accuracy: 0.7713841368584758
Random Forest n_estimators = 10 Accuracy: 0.7884914463452566
Random Forest n_estimators = 50 Accuracy: 0.7947122861586314
Random Forest n_estimators = 100 Accuracy: 0.7947122861586314
Logistic Regression Accuracy: 0.7262830482115086


Se probó tres algoritmos de clasificación: Árbol de decisión, Bosque aleatorio y Regresión logística.

* El árbol de decisión alcanzó una exactitud máxima de 0.77 con profundidad 9–10.

* El bosque aleatorio mostró mejor desempeño, llegando a 0.795 con 50 y 100 estimadores.

* La regresión logística tuvo un rendimiento inferior (0.726).
Con base en estos resultados, seleccionamos el bosque aleatorio como el modelo final para evaluación en el conjunto de prueba.

# 4. Calidad de modelo

In [24]:
best_model = RandomForestClassifier(n_estimators=100, random_state=12345)
best_model.fit(features_train, target_train)

# evaluar en test
test_predictions = best_model.predict(features_test)
test_accuracy = accuracy_score(target_test, test_predictions)

print("Exactitud en el conjunto de prueba:", test_accuracy)

Exactitud en el conjunto de prueba: 0.7838258164852255


Después de probar distintos modelos (Árbol de decisión, Bosque aleatorio y Regresión logística), el mejor desempeño se obtuvo con un Bosque Aleatorio de 100 estimadores.

* Exactitud en validación: 0.7947

* Exactitud en prueba: 0.7838

* La diferencia entre validación y prueba es pequeña, lo cual indica que el modelo generaliza adecuadamente. El umbral requerido de 0.75 fue superado.

# 5. Prueba de cordura

In [26]:
# clase más común en entrenamiento
most_common = target_train.mode()[0]

# generar predicciones constantes
baseline_predictions = np.full_like(target_test, fill_value=most_common)

# exactitud baseline
baseline_accuracy = accuracy_score(target_test, baseline_predictions)

print("Baseline (clase más frecuente):", baseline_accuracy)
print("Random Forest (mejor modelo):", test_accuracy)

Baseline (clase más frecuente): 0.6951788491446346
Random Forest (mejor modelo): 0.7838258164852255


Para verificar la utilidad del modelo, realizamos una prueba de cordura comparando contra un baseline simple: predecir siempre la clase más frecuente.

* Exactitud baseline: 0.696

* Exactitud del modelo (Random Forest): 0.784

El modelo supera claramente al baseline, lo que demuestra que es capaz de identificar patrones de comportamiento en los clientes y generalizar de forma efectiva.

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Buen trabajo con la prueba de cordura. Con esto estás validando que los resultados obtenidos en tus anteriores modelos son mejores que con un modelo trivial.

# Conclusión del Proyecto

En este proyecto se desarrolló un modelo de clasificación supervisada para predecir si un cliente de la compañía Megaline debería ser asignado al plan Smart o Ultra, con base en su comportamiento de uso de llamadas, minutos, mensajes y tráfico de internet. El dataset utilizado contenía 3,214 registros sin valores faltantes ni duplicados, y se dividió en conjuntos de entrenamiento (60%), validación (20%) y prueba (20%) para garantizar una correcta evaluación.

Se entrenaron y compararon distintos modelos: árbol de decisión, regresión logística y bosque aleatorio. El árbol de decisión alcanzó una exactitud máxima de aproximadamente 0.77 con profundidad de 9 a 10, mientras que la regresión logística obtuvo alrededor de 0.73 en validación. El bosque aleatorio, en cambio, mostró un mejor desempeño con hasta 0.795 de exactitud en validación al utilizar entre 50 y 100 árboles, por lo que fue seleccionado como modelo final.

La evaluación del modelo en el conjunto de prueba alcanzó una exactitud de 0.784, superando el umbral requerido de 0.75 y mostrando una diferencia mínima respecto al conjunto de validación, lo que confirma que el modelo generaliza adecuadamente. Además, al compararlo con un baseline que consiste en predecir siempre la clase más frecuente (exactitud de 0.696), el modelo logró una mejora de casi 9 puntos porcentuales, lo que demuestra que captura patrones reales en los datos y no se limita a replicar la clase mayoritaria.

En conclusión, el bosque aleatorio se consolidó como la mejor alternativa para este problema, alcanzando una exactitud final del 78%. El modelo cumple con el umbral establecido y aporta un valor significativo frente a estrategias triviales, por lo que puede ser utilizado por Megaline como una herramienta práctica para recomendar planes a nuevos clientes y facilitar la migración desde planes antiguos hacia los planes Smart o Ultra.